# Uncertainty Quantification for test cases

In [1]:
import os
import shutil

import numpy as np
from scipy.stats import norm,uniform

import UQLib.sample.TMCMC as TMCMC
import test_cases.sinusoidal as sinusoidal

np.random.seed(1234)

## Inverse UQ for a sinusoidal function

In [2]:
modelpath = "/home/kevin/master_project/IUQ-Project/test_cases/sinusoidal.py"

# Generate data with Gaussian noise
x, y, y_err, true_params = sinusoidal.generate(10)

model_params = ["a","b","c","k1","k2","f1","f2"]

print("True parameter values:")
print(tuple(model_params), "=")
print(tuple(true_params))

error_params = ["s"]
design_vars = ["x"]

model_errors = ["s" for n in range(x.shape[0])]

# Define uniform prior wrapper functions and samplers for the model parameters
model_func = lambda val: uniform.pdf(val,-1.0,2.0)
model_sampler = lambda n_samples: np.random.uniform(-1.0,1.0,size=n_samples)

# Define prior wrapper function and sampler for the error parameter
error_func = lambda val: uniform.pdf(val,0.001,0.999)
error_sampler = lambda n_samples: np.random.uniform(0.001,1.0,size=n_samples)

model_priors = [(model_func,model_sampler) for param in model_params]
error_priors = [(error_func,error_sampler)]

# Construct problem dict
problem = {"setup":sinusoidal.setup,
           "measure":sinusoidal.read_output,
           "modelpath":modelpath,
           "model parameters":model_params,
           "error parameters":error_params,
           "design variables":design_vars,
           "input data":x,
           "output data":y,
           "data errors":y_err,
           "error mapping":model_errors,
           "model priors":model_priors,
           "error priors":error_priors
          }

os.makedirs("TMCMC_output")
os.chdir("TMCMC_output")

samples,likelihoods = TMCMC.sample(problem,1000,nprocs=10)

os.chdir("..")

# Remove garbage
shutil.rmtree("./TMCMC_output")

print(samples)
print(likelihoods)

True parameter values:
('a', 'b', 'c', 'k1', 'k2', 'f1', 'f2') =
(-0.6169610992422154, 0.24421754207966373, -0.12454452198577104, 0.5707171674275384, 0.559951616237607, -0.45481478943471676, -0.4470714897138066)
Calculate next p
p: 0.17549954886782185
COV: 0.9999989892627099
Running stage 1...
Current max likelihood: 3.82362776466929e-06
Calculate next p
p: 0.936948206719835
COV: 0.9999988920692441
Running stage 2...
Current max likelihood: 9.248439011665014e-06
Calculate next p
p: 1
COV: 0.09586164243635655
Running stage 3...
Current max likelihood: 1.2761669668269788e-05
[[ 0.2320228  -0.03854658 -0.63802385  0.30969622  0.08000516  0.29059115
   0.0555876   0.53212547]
 [ 0.24992677 -0.16542866 -0.7208654   0.2299283   0.19696936  0.3184778
   0.08833074  0.52487514]
 [ 0.24992677 -0.16542866 -0.7208654   0.2299283   0.19696936  0.3184778
   0.08833074  0.52487514]
 [ 0.29183011 -0.24041932 -0.78517861  0.1917818   0.31756456  0.30375063
   0.10670899  0.53045496]
 [ 0.4278728  -0.3